In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
toxicWordsTrain=pd.read_csv("dataset/jigsaw-toxic-comment-classification-challenge/train.csv").fillna(' ')
toxicWordsTest=pd.read_csv("dataset/jigsaw-toxic-comment-classification-challenge/test.csv").fillna(' ')

#Class labels
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y_train = toxicWordsTrain[list_classes].values
x_train = toxicWordsTrain["comment_text"]
x_test  = toxicWordsTest["comment_text"]

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create tokenizer
tokenizer = Tokenizer(num_words=None,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      char_level=False)

# Fit and run tokenizer
tokenizer.fit_on_texts(list(x_train))
tokenized_train = tokenizer.texts_to_sequences(x_train)
tokenized_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index


# Extract variables
vocab_size = len(word_index)
longest = max(len(seq) for seq in tokenized_train)
average = np.mean([len(seq) for seq in tokenized_train])
stdev = np.std([len(seq) for seq in tokenized_train])
max_len = int(average + stdev * 3)


In [8]:
# Pad sequences
processed_X_train = pad_sequences(tokenized_train, maxlen=max_len, padding='post', truncating='post')
processed_X_test = pad_sequences(tokenized_test, maxlen=max_len, padding='post', truncating='post')

In [9]:
print(processed_X_train.shape)
print(y_train.shape)

(159571, 371)
(159571, 6)


In [10]:
import tensorflow as tf
model = tf.keras.models.load_model("model.best.hdf5")

In [11]:
def bar_chart(prediction):
    # Prepare data for bar chart
    labels = ['Toxic', 'Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate']
    values = prediction[0]  # Assuming prediction returns probabilities for each category

    # Define colors for each bar
    colors = ['skyblue', 'lightcoral', 'yellowgreen', 'gold', 'violet', 'orange']

    # Create bar chart
    plt.figure(figsize=(10, 5))
    bars = plt.bar(labels, values, color=colors)

    # Add text annotations for each bar
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval * 100:.0f}%', ha='center', va='bottom')

    # Customize y-axis ticks and labels
    plt.yticks(np.arange(0, 1.1, 0.1), [f'{int(i * 100)}%' for i in np.arange(0, 1.1, 0.1)])
    plt.ylabel('Probability')
    plt.title('Toxicity Levels')

    # Save the plot to a file
    plt.savefig('toxicity_levels.png')
    plt.close()

    return 'toxicity_levels.png'

In [12]:

def toxicity_level(string):
    """
    Return toxicity probability based on inputed string.
    """
    # Process string
    new_string = [string]
    new_string = tokenizer.texts_to_sequences(new_string)
    new_string = pad_sequences(new_string, maxlen=max_len, padding='post', truncating='post')
    
    # Predict
    prediction = model.predict(new_string)
    
    # Print output
    response = f"Toxicity levels for '{string}': \n" \
                f"Toxic:         {prediction[0][0]:.0%} \n" \
                f"Severe Toxic:  {prediction[0][1]:.0%} \n" \
                f"Obscene:       {prediction[0][2]:.0%} \n" \
                f"Threat:        {prediction[0][3]:.0%} \n" \
                f"Insult:        {prediction[0][4]:.0%} \n" \
                f"Identity Hate: {prediction[0][5]:.0%} \n"
    
    bar_chart_file = bar_chart(prediction)

    return response, bar_chart_file

In [13]:
print(toxicity_level("stupid")[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
Toxicity levels for 'stupid': 
Toxic:         92% 
Severe Toxic:  0% 
Obscene:       57% 
Threat:        0% 
Insult:        68% 
Identity Hate: 0% 



In [14]:
print(toxicity_level("dumb")[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Toxicity levels for 'dumb': 
Toxic:         71% 
Severe Toxic:  0% 
Obscene:       24% 
Threat:        0% 
Insult:        42% 
Identity Hate: 0% 



In [15]:
print(toxicity_level("Dumb Shit")[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Toxicity levels for 'Dumb Shit': 
Toxic:         99% 
Severe Toxic:  2% 
Obscene:       94% 
Threat:        0% 
Insult:        66% 
Identity Hate: 0% 



In [ ]:
import gradio as gr
interface = gr.Interface(
    fn = toxicity_level,
    inputs = gr.Textbox(label="Toxic Comment Detection for:", lines=2, placeholder="Enter your comment here..."),
    outputs = [gr.Textbox(label="Statistic", lines=3), gr.Image(label="Bar Chart", type="filepath")],
    title = "Toxic Comment Detection",
    description = "Enter a comment to check if it's toxic or not."
)

# Launch the interface
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://1a7106c8a789c0a4d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
